In [39]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text

db_eng = create_engine('sqlite:////Users/hj/Dropbox/real_estate/data/db/RealEstate.db')

with db_eng.connect() as conn:
    result = conn.execute(text("SELECT name FROM sqlite_master WHERE type = 'table' "))
    for row in result:
        print(row)

('apt_lease_raw',)
('house_lease_raw',)
('office_lease_raw',)
('multi_family_lease_raw',)
('apt_raw',)
('commerical_raw',)
('multi_family_raw',)
('house_raw',)


In [46]:
import pandas as pd

apt= pd.read_sql('SELECT * FROM apt_raw LIMIT 100', db_eng)
commercial= pd.read_sql('SELECT * FROM commerical_raw LIMIT 100', db_eng)
house = pd.read_sql('SELECT * FROM house_raw LIMIT 100', db_eng)


house_lease = pd.read_sql('SELECT * FROM house_lease_raw LIMIT 100', db_eng)
apt_lease = pd.read_sql('SELECT * FROM apt_lease_raw LIMIT 100', db_eng)
office_lease = pd.read_sql('SELECT * FROM office_lease_raw LIMIT 100', db_eng)
multi_family_lease = pd.read_sql('SELECT * FROM multi_family_lease_raw LIMIT 100', db_eng)

In [47]:
df_list = [apt, house_lease, apt_lease]
col_list = []
for df in df_list :
    col_list.append(df.columns.tolist())


dfs_lease = {'상업용' : commercial, '단독다가구' : house, '아파트' : apt}
dfs = {'아파트_임대' : apt_lease, '단독다가구_임대' : house_lease, '오피스텔_임대' : office_lease, '연립다세대_임대' : multi_family_lease}



data_frames = dfs
all_columns = set(col for df in data_frames.values() for col in df.columns)

# 각 데이터 프레임에 대해 해당 칼럼이 존재하는지 체크하여 데이터 프레임 생성
column_presence_df = pd.DataFrame(
    {col: [1 if col in df.columns else 0 for df in data_frames.values()] for col in all_columns},
    index=data_frames.keys()
)

column_presence_df

,일,시군구,매수자,보증금액,층,건축년도,갱신요구권사용,종전계약월세,해제사유발생일,단지,...,거래유형,지번,중개사소재지,월세금액,계약면적,년,전용면적,종전계약보증금,매도자,계약구분
아파트_임대,1,0,0,1,1,1,1,1,0,0,...,0,1,0,1,0,1,1,1,0,1
단독다가구_임대,1,0,0,1,0,1,1,1,0,0,...,0,0,0,1,1,1,0,1,0,1
오피스텔_임대,1,1,0,0,1,1,1,1,0,1,...,0,1,0,0,0,1,1,1,0,1
연립다세대_임대,1,0,1,0,1,1,0,0,1,0,...,1,1,1,0,0,1,1,0,1,0


In [50]:
# 모든 데이터 프레임의 칼럼을 추출하여 유니크한 칼럼 리스트 생성
all_columns = set(col for df in data_frames.values() for col in df.columns)

# 각 데이터 프레임에 대해 해당 칼럼이 존재하는지 체크하여 데이터 프레임 생성
column_presence_df = pd.DataFrame(
    {col: [1 if col in df.columns else 0 for df in data_frames.values()] for col in all_columns},
    index=data_frames.keys()
)

# 각 칼럼이 모든 데이터 프레임에서 존재하는지 확인
common_columns = [col for col in all_columns if column_presence_df[col].sum() == len(data_frames)]

print("모든 데이터 프레임에 공통으로 존재하는 칼럼들:", data_frames.keys(), common_columns)

모든 데이터 프레임에 공통으로 존재하는 칼럼들: dict_keys(['아파트_임대', '단독다가구_임대', '오피스텔_임대', '연립다세대_임대']) ['일', '건축년도', '지역코드', '월', '법정동', '년']


In [ ]:
apt.drop(columns = ['거래유형', '도로명지상지하코드', '중개사소재지', '해제사유발생일', '해제여부'])